In [1]:
from collections import defaultdict

In [2]:
from functools import lru_cache

In [3]:
prices = defaultdict(lambda: -float('inf'))

In [4]:
for i, v in enumerate([1, 5, 8, 9, 10, 17, 17, 20, 24, 30]):
    prices[i + 1] = v

In [5]:
solution = {}

@lru_cache(500)
def revenue(r):
    split, r_star = max([(0, prices[r])] + [(i, revenue(i) + revenue(r - i)) for i in range(1, r)], key=lambda x: x[1])
    solution[r] = split, r - split
    return r_star

In [6]:
revenue(100)

300

In [7]:
def parse_solution(r, solution):
    left, right = solution[r]
    if left == 0:
        return [right]
    return [left] + parse_solution(right, solution)
    

In [8]:
def pretty_solution(splits):
    return ' -> '.join(map(str, splits))

In [9]:
pretty_solution(parse_solution(57, solution))

'1 -> 6 -> 10 -> 10 -> 10 -> 10 -> 10'

In [10]:
from functools import wraps

In [11]:
def memo(func):
    cache = {}
    @wraps(func)
    def wrapper(*args, **kwargs):
        str_key = str(args) + str(kwargs)
        if str_key not in cache:
            cache[str_key] = func(*args, **kwargs)
        return cache[str_key]
    return wrapper

In [12]:
@memo
def get_edit_distance(string1, string2):
    if len(string1) == 0:
        return len(string2)
    if len(string2) == 0:
        return len(string1)
    
    return min(
        [get_edit_distance(string1[:-1], string2) + 1,
         get_edit_distance(string1, string2[:-1]) + 1,
         get_edit_distance(string1[:-1], string2[:-1]) + (0 if string1[-1] == string2[-1] else 2)]
    )

In [13]:
get_edit_distance('biejing', 'beijing')

2

In [14]:
get_edit_distance('biejing', 'beijie')

5

In [38]:
@memo
def get_edit_distance_trace(string1, string2, solution):
    if len(string1) == 0:
        return len(string2)
    if len(string2) == 0:
        return len(string1)
    
    come_from1, come_from2, result =  min(
        [(len(string1)-1, len(string2), get_edit_distance_trace(string1[:-1], string2, solution) + 1),
         (len(string1), len(string2)-1, get_edit_distance_trace(string1, string2[:-1], solution) + 1),
         (len(string1)-1, len(string2)-1, get_edit_distance_trace(string1[:-1], string2[:-1], solution) + (0 if string1[-1] == string2[-1] else 2))],
        key=lambda x: x[2]
    )
    solution[len(string1), len(string2)] = come_from1, come_from2, result
    return result



In [39]:
def parse_solution(string1, string2, solution, result):
    cur = string1
    s = [cur]
    l1, l2, r = len(string1), len(string2), result
    while l1 > 0 or l2 > 0:
        newl1, newl2, r = solution[l1, l2]
        if r == 0:
            return s
        if newl1 < l1 and newl2 < l2:
            pass
        elif newl1 < l1:
            cur = cur[:newl1] + cur[newl1+1:]
            s.append(cur)
        else:
            cur = cur[:newl2] + string2[newl2] + cur[newl2:]
            s.append(cur)
        l1, l2 = newl1, newl2
    return s

In [40]:
solution1 = {}
word1 = 'beijinghello'
word2 = 'biejieehll'
result = get_edit_distance_trace(word1, word2, solution1)
print(result)
sol = parse_solution(word1, word2, solution1, result)
pretty_solution(sol)

8


'beijinghello -> beijinghell -> beijinghll -> beijinhll -> beijihll -> beijihell -> beijiehell -> bejiehell -> biejiehell'